In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,27980
2,Huelva,Confirmados PDIA 14 días,1288
3,Huelva,Tasa PDIA 14 días,"250,98895102987314"
4,Huelva,Confirmados PDIA 7 días,577
5,Huelva,Total Confirmados,28153
6,Huelva,Curados,23528
7,Huelva,Fallecidos,362


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  27980.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7242.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1199 personas en los últimos 7 días 

Un positivo PCR cada 517 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,27980.0,577.0,1288.0,513170.0,112.438373,250.988951,127.0
Condado-Campiña,8695.0,316.0,677.0,156231.0,202.264595,433.332693,84.0
Huelva-Costa,16370.0,190.0,480.0,289548.0,65.619517,165.775623,24.0
Almonte,1438.0,108.0,249.0,24507.0,440.690415,1016.036235,20.0
Sierra de Huelva-Andévalo Central,2503.0,63.0,113.0,67391.0,93.484293,167.678177,16.0
Huelva (capital),7242.0,120.0,278.0,143837.0,83.427769,193.274331,15.0
San Juan del Puerto,432.0,19.0,26.0,9411.0,201.891404,276.272447,9.0
Bollullos Par del Condado,773.0,36.0,63.0,14387.0,250.225898,437.895322,9.0
Bonares,242.0,17.0,23.0,6060.0,280.528053,379.537954,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cumbres de San Bartolomé,10.0,5.0,5.0,376.0,1329.787234,1329.787234,2.0
Cumbres Mayores,48.0,21.0,23.0,1749.0,1200.686106,1315.037164,5.0
Almonte,1438.0,108.0,249.0,24507.0,440.690415,1016.036235,20.0
Santa Ana la Real,9.0,3.0,4.0,475.0,631.578947,842.105263,0.0
Rociana del Condado,495.0,27.0,64.0,7939.0,340.093211,806.146870,4.0
Lucena del Puerto,202.0,13.0,20.0,3261.0,398.650721,613.308801,8.0
Galaroza,24.0,7.0,8.0,1382.0,506.512301,578.871201,0.0
Cartaya,1240.0,27.0,114.0,20083.0,134.442065,567.644276,1.0
Beas,185.0,11.0,20.0,4341.0,253.397835,460.723336,7.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Almonaster la Real,63.0,1.0,8.0,1788.0,55.928412,447.427293,0.0,0.125000
Villarrasa,147.0,1.0,7.0,2211.0,45.228403,316.598824,0.0,0.142857
Palos de la Frontera,569.0,10.0,43.0,11742.0,85.164367,366.206779,4.0,0.232558
Cartaya,1240.0,27.0,114.0,20083.0,134.442065,567.644276,1.0,0.236842
Ayamonte,1263.0,2.0,7.0,21104.0,9.476876,33.169067,1.0,0.285714
Minas de Riotinto,109.0,3.0,9.0,3812.0,78.698846,236.096537,2.0,0.333333
San Bartolomé de la Torre,282.0,5.0,14.0,3761.0,132.943366,372.241425,0.0,0.357143
Lepe,2064.0,8.0,22.0,27880.0,28.694405,78.909613,0.0,0.363636
Aracena,506.0,7.0,18.0,8255.0,84.797093,218.049667,1.0,0.388889
